In [ ]:
# Initiate important libraries
import pandas as pd
import os
import sys
from pycaret.regression import *
from sklearn.model_selection import train_test_split

In [ ]:
# Define the meta model generation
def meta_model_generator (chemblid, fingerprint_list, model_id = 'lightgbm', meta_model = 'lr'):
    for x in chemblid:
        recap_df = pd.DataFrame()
        metrics_df = pd.DataFrame()
        path = f'meta_models/metrics'
        isExist = os.path.exists(path)
        if not isExist:
        # Create a new directory because it does not exist 
            os.makedirs(path)
        path = f'meta_models/input'
        isExist = os.path.exists(path)
        if not isExist:
        # Create a new directory because it does not exist 
            os.makedirs(path)
        df = pd.read_csv(f'fingerprints\{x}_{fingerprint_list[0]}.csv')
        df_train, df_test = train_test_split(df, test_size = 0.30, random_state = 42)
        reg = setup(df_train, target='pIC50', session_id=42, use_gpu=True, train_size=0.7, normalize=True, normalize_method='minmax', silent=True)
        model = create_model(model_id)
        metrics = pull()
        final_model = finalize_model(model)
        final_target = df_test['pIC50']
        df_predict = predict_model(final_model, data=df_test)
        metrics_df[f'{fingerprint_list[0]}'] = metrics['R2']
        recap_df[f'{fingerprint_list[0]}'] = df_predict['Label']

        df = pd.read_csv(f'fingerprints\{x}_{fingerprint_list[1]}.csv')
        df_train, df_test = train_test_split(df, test_size = 0.30, random_state = 42)
        reg = setup(df_train, target='pIC50', session_id=42, use_gpu=True, train_size=0.7, normalize=True, normalize_method='minmax', silent=True)
        model = create_model(model_id)
        metrics = pull()
        final_model = finalize_model(model)
        final_target = df_test['pIC50']
        df_predict = predict_model(final_model, data=df_test)
        metrics_df[f'{fingerprint_list[1]}'] = metrics['R2']
        recap_df[f'{fingerprint_list[1]}'] = df_predict['Label']
        recap_df['pIC50'] = final_target
        recap_df.to_csv(f'meta_models/input/{x}_meta_input_{fingerprint_list[0]}_{fingerprint_list[1]}_{model_id}.csv')

        for m in meta_model:
            reg = setup(recap_df, target='pIC50', session_id=42, use_gpu=True, train_size=0.7, normalize=True, normalize_method='minmax', silent=True)
            meta = create_model(m)
            meta_tuned = tune_model(meta, n_iter=25, choose_better= True)
            metrics = pull()
            metrics_df[f'{m}'] = metrics['R2']
        metrics_df.to_csv(f'meta_models/metrics/{x}_meta_metrics_{fingerprint_list[0]}_{fingerprint_list[1]}.csv', index=False)

In [ ]:
# Initiate important variables
target_id = [
    'CHEMBL238', 
    'CHEMBL218', 
    'CHEMBL3650', 
    'CHEMBL220', 
    'CHEMBL248', 
]
fingerprints = [
    'ECFP6_2048',
    # 'mol2vec',
    # 'pubchem',
    'pairfps',
    # 'rdk'
]
# Run the meta model generator. Currently this function only accepts exactly one pair of fingerprints, and therefore must be run several times
meta_model_generator(target_id, fingerprints)